# Game theory

Author : Philippe Mathieu, [CRISTAL Lab](http://www.cristal.univ-lille.fr), [SMAC team](https://www.cristal.univ-lille.fr/?rubrique27&eid=17), [Lille University](http://www.univ-lille.fr), email : philippe.mathieu@univ-lille.fr

Contributors : Louisa Fodil (CRISTAL/SMAC), Céline Petitpré (CRISTAL/SMAC)

Creation : 2018-01-18


## Introduction

Game theory is the discipline that formally studies the interactions that can occur between several individuals in a strategic situation. The term "game" is not a priori a playful term, Von Neumann having developed it for the modelling of conflict interactions. In particular, it has been used to model and simplify many problems ranging from trade relations to armed conflicts. Game theory decomposes games according to the number of players (usually 2), then mainly into two main families: simultaneous games, in which players play simultaneously and therefore do not know when playing what the opponent is playing (for example, Rock Paper Scissors) and non-concurrent games where everyone plays in turn (chess for example). 

We are here in the context of the **two-player, simultaneous** games. This kind of game is usually represented by a payoff matrix. The moves of one of the two players are on the abscissa and those of the other on the ordinate. The intersection of each square characterizes a game situation. It contains the points that will be distributed to each of the two players. For information, game theory further decomposes these games into two other families: zero-sum games in which everything lost by one is won by the other, and non-zero-sum games in which the winner can win more points than what is lost by the loser.

This sheet aims to show how to represent and generate games, then how to calculate the different equilibriums of a game.

## A payoff matrix

First of all, it is necessary to be able to code a payoff matrix. A `Game` class will allow us to store the winnings of both players for a given situation. 
A Game object takes as parameter an array of pairs of integers corresponding to the scores of each outcome of the game, as well as a table of the names of the corresponding actions.
Internally, the *numpy* package offers an ideal `Array` object to store and manipulate this. For a game with `n` choice options for each player, we create a `n*n` array of pairs of values `(x,y)` with `x` the win of player 1 and `y` the win of player 2.


In [ ]:
import numpy as np
import math
import pandas as pd


class Game:
    def __init__(self, tab, actions):
        self.actions = actions
        m = np.array(tab, dtype=[("x", object), ("y", object)])
        self.size = int(math.sqrt(len(tab)))
        self.scores = m.reshape(self.size, self.size)

    def prettyPrint(self):
        game = pd.DataFrame(np.nan, self.actions, self.actions, dtype=object)
        for i in range(self.size):
            for j in range(self.size):
                game.iat[i, j] = self.scores[i][j]
        print(game)


print("All is ok")

In [ ]:
# Some famous games

# battle of the sexes
gs=[(3,2),(1,1),(0,0),(2,3)]     

# chicken game
cg=[(-100,-100),(2,-2),(-2,2),(0,0)]     

# matching pennies
mp=[(1,-1),(-1,1),(-1,1),(1,-1)] 

# rock-paper-scissors
rpc=[(0,0),(-1,1),(1,-1),(1,-1),(0,0),(-1,1),(-1,1),(1,-1),(0,0)] 

# prisoner's dilemma
dp =[(3,3),(0,5),(5,0),(1,1)]
g = Game(dp,['C','D'])
g.prettyPrint()

# The equilibrium notion

Economists very quickly took advantage of these models to study different economic problems. In order to identify the best way to play a fixed game, they based themselves on the abstract notion of player rationality. Assuming that, in the worst case scenario, the opponent is also rational, the reasoning I apply must therefore also be applied to my opponent. Very quickly, several definitions of rationality appeared, thus defining one (or possibly several) fixed point of a game, point towards which rationally, if everyone thinks in the same way, we will reach:
1. *The Nash equilibrium*, from the name of its inventor, [John Forbes Nash](https://fr.wikipedia.org/wiki/John_Forbes_Nash). Nash equilibrium is a situation of no regret for each of the players. *"we played simultaneously, but now that I know what the other one played, I have no regrets "*
2. *The Pareto optimum*, from the name of its inventor, [Vilfredo Pareto](https://fr.wikipedia.org/wiki/Vilfredo_Pareto). The Pareto Optimum is a situation where no situation is superior for both players, and therefore they have no collective interest in changing.
3. *The dominant strategy equilibrium*. A "rational" player will never play one of his strategies if it is dominated by another of his strategies. We can therefore "simplify" a game by iteratively eliminating the dominated strategies.



## The Nash equilibrium

To compute the "no regret" situation corresponding to the Nash equilibrium, it is sufficient to note the answers of player 1 most adapted to the possible choices of player 2 (therefore calculate the max(x) in each column), then calculate the best answers of player 2 to the possible choices of player 1 (therefore calculate the max(y) in each line). If an issue has 2 max, then it is a Nash equilibrium. Depending on the games, there may of course be 1, several or none at all. 
The use of a `np.Array` makes things much easier since it is possible to have the max value vectors in line or in column. It is therefore sufficient to create Boolean matrices in each case and compute a `or` logical on them.

In [ ]:
def getNash(self):
    max_x = np.matrix(self.scores["x"].max(0)).repeat(self.size, axis=0)
    bool_x = self.scores["x"] == max_x
    max_y = np.matrix(self.scores["y"].max(1)).transpose().repeat(self.size, axis=1)
    bool_y = self.scores["y"] == max_y
    bool_x_y = bool_x & bool_y
    result = np.where(bool_x_y == True)
    listOfCoordinates = list(zip(result[0], result[1]))
    return listOfCoordinates


print("All is ok")

#### Application to the battle of sexes
In [the battle of sexes](https://en.wikipedia.org/wiki/Battle_of_the_sexes_(game_theory)) a couple has made an appointment for the evening, but none of them can remember whether it is to attend a football match or go to the opera. 
The husband would rather go to see football, the wife would like to go to the opera. 
However, they both prefer to go to the same place rather than be alone.
If the man goes to the stadium and his wife is there, he gets 3.
If he is alone at the stadium, he gets 1.
If he goes to the opera and his wife is at the stadium, he gets 0.
If he goes to the opera and his wife is also in the opera, he gets 2.
(Same thing for the woman by reversing opera and stadium)
We thus obtain the following matrix `gs` on which we can test the Nash equilibrium.

In [ ]:
gs = [(3, 2), (1, 1), (0, 0), (2, 3)]
g = Game(gs, ["Opera", "Stadium"])

# We recover the indexes of the Nash equilibrium(s)
listOfCoordinates = getNash(g)
print("The indexes of Nash's equilibrium : ")
print(listOfCoordinates)

# We print the moves corresponding to these equilibrium
print("The corresponding rounds : ")
for cor in listOfCoordinates:
    print(g.actions[cor[0]], g.actions[cor[1]])

# We print the corresponding scores
print("The corresponding scores : ")
for cor in listOfCoordinates:
    print(g.scores[cor[0]][cor[1]])

We notice here that the game has two Nash equilibria that correspond to situations where the couple is together at the same place.

## Exercise 1

In [ ]:
# Find find Nash's equilibrium(s) (if any exist) for the rock-paper-scissors game 

rpc=[(0,0),(-1,1),(1,-1),(1,-1),(0,0),(-1,1),(-1,1),(1,-1),(0,0)]
g = Game(rpc,['R','P','C'])

# Same for the iterated prisoner's dilemma

## The Pareto optimum
The Pareto Optimum corresponds to issues where both players have no collective interest in changing. For each of the game issues, it is ensured that no other issue increases both Player 1's and Player 2's scores.

In [ ]:
def isPareto(self, t, s):
    return (
        True
        if (len(s) == 0)
        else (s[0][0] <= t[0] or s[0][1] <= t[1]) and isPareto(self, t, s[1:])
    )


def getPareto(self):
    x = 0
    y = 0
    res = list()
    liste = self.scores.flatten()
    for s in liste:
        if x == self.size:
            x = 0
            y = y + 1
        if isPareto(self, s, liste):
            res.append((x, y))
        x = x + 1
    return res


print("All is ok")

#### Let's continue the battle of sexes game

In [ ]:
gs=[(3,2),(1,1),(0,0),(2,3)]
g = Game(gs,['Opera','Stadium'])

# We recover the indexes of the pareto optimum(s)
listOfCoordinates = getPareto(g)
print("The indexes of Pareto's optimums : ")
print(listOfCoordinates)

# We print the moves corresponding to these optimums
print("The corresponding rounds : ")
for cor in listOfCoordinates : 
    print(g.actions[cor[0]], g.actions[cor[1]])

# We print the corresponding scores
print("The corresponding scores : ")
for cor in listOfCoordinates : 
    print(g.scores[cor[0]][cor[1]])


We notice that there are two Pareto optima for the battle of sexes that are the same as for Nash's equilibria.

## Exercise 2

In [ ]:
# Find find Pareto's equilibrium(s) (if any exist) for the rock-paper-scissors game 

rpc=[(0,0),(-1,1),(1,-1),(1,-1),(0,0),(-1,1),(-1,1),(1,-1),(0,0)]
g = Game(rpc,['R','P','C'])

# Same for the iterated prisoner's dilemma

## The dominant strategy equilibrium

For a rational player, if one of his strategies is dominated by another for all the opponent's responses, then it has no interest. The game can therefore be simplified by eliminating these different strategies in turn (EISD). If there is only one issue left after this process, this issue is called the equilibrium in dominant strategy. An equilibrium in dominant strategies is always a Nash equilibrium but the opposite is not true.

If we have eliminated the *weakly* dominated strategies and there is only one strategy left for each player, then it is a Nash equilibrium, but there may be others (and depending on the order of elimination we find one or another)

In [ ]:
def getDominantStrategies(self, strict="True"):
    dominatedLines = []
    dominatedColumns = []
    findDominated = True
    while (
        findDominated
        and (len(dominatedLines) != self.size - 1)
        and (len(dominatedColumns) != self.size - 1)
    ):
        findDominated = False
        # We look at dominated lines
        for i in range(self.size - 1):
            line1 = self.scores["x"][i]
            line2 = self.scores["x"][i + 1]
            if compare(self, line1, line2, dominatedColumns, strict):
                if i not in dominatedLines:
                    dominatedLines += [i]
                    findDominated = True
            if compare(self, line2, line1, dominatedColumns, strict):
                if i + 1 not in dominatedLines:
                    dominatedLines += [i + 1]
                    findDominated = True
        # We look at dominated columns
        for i in range(self.size - 1):
            c1 = self.scores["y"].transpose()[i]
            c2 = self.scores["y"].transpose()[i + 1]
            if compare(self, c1, c2, dominatedLines, strict):
                if i not in dominatedColumns:
                    dominatedColumns += [i]
                    findDominated = True
            if compare(self, c2, c1, dominatedLines, strict):
                if i + 1 not in dominatedColumns:
                    dominatedColumns += [i + 1]
                    findDominated = True
    return result(self, dominatedLines, dominatedColumns)


def compare(self, l1, l2, tab, strict):
    dominated = True
    for i in range(self.size):
        if strict:
            if (l1[i] < l2[i] and i not in tab) or i in tab:
                dominated = dominated and True
            else:
                dominated = dominated and False
        else:
            if (l1[i] <= l2[i] and i not in tab) or i in tab:
                dominated = dominated and True
            else:
                dominated = dominated and False
    return dominated


def result(self, dominatedLines, dominatedColumns):
    x = list()
    y = list()
    res = list()

    for i in range(self.size):
        if i not in dominatedLines:
            x.append(i)
        if i not in dominatedColumns:
            y.append(i)

    for indX in x:
        for indY in y:
            res.append((indX, indY))

    return res


print("All is ok")

#### Let's continue with the battle of sexes game. 
Strictly and weakly dominated strategies are tested here.

In [ ]:
gs=[(3,2),(1,1),(0,0),(2,3)]     
g = Game(gs,['Opera','Stadium'])


# We recover the indexes of the non-dominated strategy(ies):
print("Strongly dominated strategies method")
listOfCoordinates = getDominantStrategies(g)
print("Non-dominated strategies indexes : ")
print(listOfCoordinates)

# We print the moves corresponding to these strategies
print("The corresponding rounds : ")
for cor in listOfCoordinates : 
    print(g.actions[cor[0]], g.actions[cor[1]])

# We print the corresponding scores
print("The corresponding scores : ")
for cor in listOfCoordinates : 
    print(g.scores[cor[0]][cor[1]])

print(" ")

# We recover the indexes of the non-dominated strategy(ies)::
print("Weakly dominated strategies method")
listOfCoordinates = getDominantStrategies(g, strict="False")
print("Non-dominated strategies indexes : ")
print(listOfCoordinates)

# We print the moves corresponding to these strategies
print("The corresponding rounds : ")
for cor in listOfCoordinates : 
    print(g.actions[cor[0]], g.actions[cor[1]])

# We print the corresponding scores
print("The corresponding scores : ")
for cor in listOfCoordinates : 
    print(g.scores[cor[0]][cor[1]])

## Exercise 3

In [ ]:
# Simplify the following abstract game. Is there an equilibrium in dominant strategies?
exo=[(3,6),(7,1),(4,8),(5,1),(8,2),(6,1),(6,0),(6,2),(3,2)]
g = Game(exo,['A','B','C'])

# the answer must be a single issue (B,B)->(8,2)
listOfCoordinates = getDominantStrategies(g)
print(listOfCoordinates)


## Integration in the class Game
All these methods can of course be integrated into the Game class.


### Game(tab, actions)
- `tab` : the list of score pairs
- `actions` : the list of possible choices


#### Methods
- `getDominantStrategies(self, strict='True')` which prints a list of indexes of non-dominated choices and returns a new Game with this new matrix
- `getNash(self)` which returns a list of Nash equilibrium indexes.
- `getPareto(self)` which returns a list of indexes of Pareto's equilibria

In [ ]:
%run ../src/game.py

print("All is ok")

### The prisoner's dilemma

The Prisoner's Dilemma, identified by M. Flood and M. Dresher of the Rand Corporation in 1950, is a game theory model specially created to show that Nash's equilibrium is not always a good idea.


#### Let's test the Nash and Pareto equilibriums, and the elimination of strictly dominated strategies for this game.


In [ ]:
g = Game(dp,['C','D'])


print("There is one Nash balance for the prisonner's dilemma")
listOfCoordinates = g.getNash()
for cor in listOfCoordinates : 
    print(g.actions[cor[0]], g.actions[cor[1]])
print(" ")

print("There are three Pareto optimas for the prisoner's dilemma")
listOfCoordinates = g.getPareto()
for cor in listOfCoordinates : 
    print(g.actions[cor[0]], g.actions[cor[1]])
print(" ")

print("The strictly dominant strategy for the prisoner's dilemma is the strategy where both players choose to Defect.")
listOfCoordinates = getDominantStrategies(g)
for cor in listOfCoordinates : 
    print(g.actions[cor[0]], g.actions[cor[1]])
print(" ")


## Let's have fun

### Create a random game matrix

In [ ]:
x = np.random.randint(0, 5, (3,3))
y = np.random.randint(0, 5, (3,3))
couples = [(a,b) for a,b in zip(x.flatten(),y.flatten())]
g = Game(couples, None)
print(g.scores)

## Exercise 4

In [ ]:
# Find the Nash equilibrium, Pareto optimum and the dominated strategy for this game


### Generate all possible matrices

In Python it is easy to list all the games with 2 rounds from fixed values. For example, all the games that can be built with the values 1 and 2.
The `Itertools` Library provides many efficient iterators, especially for combinations, permutations and Cartesian products. Here it is the Cartesian product of values that interests us.
We can then count for example how many games have 0, 1 or more Nash equilibria.


In [ ]:
import itertools;
import random;

def numberOfGames(valeurs, nbCoups):
    return len(valeurs)**((nbCoups**2)*2)

print("Number of games : ", numberOfGames([1,2],2))

def enumAllGames(valeurs, nbCoups):
    res = [q for q in itertools.product([p for p in itertools.product(list(valeurs), repeat=2)], repeat=nbCoups**2)]
    return [[res[j][k] for k in range(nbCoups**2)] for j in range(len(res))]

n = enumAllGames([1,2],2)
print("Printing of 10 possible random sets in this set")
for i in range (10):
    print(random.choice(n))


# Find games with particular constraints
It is now easy to search for games that have particular constraints. For example, how many 2-choices games built on (1, 2) have x Nash equilibria?

In [ ]:
def countNashEquilibria(valeurs, coups):
    results = [len(Game(i, None).getNash()) for i in enumAllGames(valeurs, coups)]
    return dict((i, results.count(i)) for i in set(results))


# How many 2-choices games built with (1, 2) have x Nash equilibria?
countNashEquilibria([1, 2], 2)

What are the two-choices games whose values are taken in (1, 2) and which have exactly the same nash and pareto equilibria?

In [ ]:
nbCoups=2
res = [q for q in itertools.combinations([p for p in itertools.product([0,1,3,5], repeat=2)], nbCoups**2)]
games = [[res[j][k] for k in range(nbCoups**2)] for j in range(len(res))]
print(str(len(games))+" games studied.")
r = []
for g in games:
    if ((sorted(Game(g,None).getPareto()) == sorted(Game(g,None).getNash())) and (len(set(g)) == len(g))):
        r.append(g)
r

## Exercise 5

In [ ]:
# Identify a game that has 1 pareto optimum, 2 Nash equilibria, but no equilibrium in dominant strategy



# Bibliographie

- John Von Neumann and Oscar Mogenstern. *Theory of Games and Economic Behavior*. Princeton Classic Editions
- Ken Binmore. *La Théorie des jeux, Une introduction.* Les éditions Arkhé
- Bernard Guerrien. *La théorie des jeux*. Economica
- Jean-Louis Boursin. *Initiation à la théorie des jeux*.  MontChretien
- [Wikipedia:GameTheory](https://en.wikipedia.org/wiki/Game_theory)